In [12]:
import pandas as pd

# Load AUTO-TOPAS

In [13]:
topas_df = pd.read_csv("./final/aspect_ratio.txt", names=["Sample ID","Rwp", "00n Size", "Isotropic Size", "Aspect Ratio"])
topas_df["Sample ID"] = topas_df["Sample ID"].str.replace(".raw", "")
topas_df["Sample ID"] = topas_df["Sample ID"].str.replace("2EXP", "4EXP")
topas_df["Sample ID"] = topas_df["Sample ID"].str.replace("3EXP", "4EXP")
topas_df.set_index("Sample ID", inplace=True)
topas_df.tail(12)

,Rwp,00n Size,Isotropic Size,Aspect Ratio
Sample ID,,,,
4EXP88,19.77,24.48,9.34,2.62
4EXP89,17.97,26.23,10.32,2.54
4EXP90,21.40,25.81,10.52,2.45
4EXP91,53.18,74.21,7.15,10.38
4EXP92,20.88,25.65,10.49,2.44
4EXP93,55.36,67.65,6.37,10.62
4EXP94,20.45,24.97,10.12,2.47
4EXP95,54.36,12.93,6.37,2.03
4EXP96,33.99,17.83,12.10,1.47


# Load XY data

In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# change resolution of plot
from matplotlib import rcParams
rcParams["font.size"] = 12
rcParams["font.family"] = "serif"
plt.rcParams['figure.dpi'] = 400
plt.rcParams["mathtext.fontset"] = "stix"

# Directory containing your raw files
directory = "./final/xy/"

# List all raw files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.xy')]  # Adjust the extension if necessary

# Analyzing each file
dict_xy = {
    "2theta": [],
}

file = files[0]
file_path = os.path.join(directory, file)
df_ = pd.read_csv(file_path, sep="\s+", skiprows=1, names=["2theta", "intensity"], dtype=float)
dict_xy["2theta"].extend(df_["2theta"])

thetha2 = []
for file in files:
    file_path = os.path.join(directory, file)
    df_ = pd.read_csv(file_path, sep="\s+", skiprows=1, names=["2theta", "intensity"], dtype=float)
    file_name = file.split("_exported.")[0]
    dict_xy[file_name] = df_["intensity"].tolist()
    
df_xy = pd.DataFrame(dict_xy).set_index("2theta")

# rename df_xy columns to 4EXP
df_xy.columns = df_xy.columns.str.replace("2EXP", "4EXP")
df_xy.columns = df_xy.columns.str.replace("3EXP", "4EXP")

# df_xy.to_excel("./final/xy/xy_bg_sub.xlsx")


# [35.496 ° - 36.859 °]
# [34.012 ° - 34.648 °]
# [31.015 ° - 32.226 °]
# [10.001 ° - 20.084 °]
#calculate the max intensity in each range

df_maxs = df_xy[(df_xy.index >= 35.496) & (df_xy.index <= 36.859)].max(axis=0).to_frame().reset_index().rename(columns={"index": "Sample ID", 0: "Int. 1"}) 
df_maxs["Int. 2"] = df_xy[(df_xy.index >= 34.012) & (df_xy.index <= 34.648)].max(axis=0).to_frame().reset_index()[0]
df_maxs["Int. 3"] = df_xy[(df_xy.index >= 31.015) & (df_xy.index <= 32.226)].max(axis=0).to_frame().reset_index()[0]
df_maxs["Int. 4"] = df_xy[(df_xy.index >= 10.001) & (df_xy.index <= 20.084)].max(axis=0).to_frame().reset_index()[0]

df_maxs["2theta 1"] = df_xy[(df_xy.index >= 35.496) & (df_xy.index <= 36.859)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 2"] = df_xy[(df_xy.index >= 34.012) & (df_xy.index <= 34.648)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 3"] = df_xy[(df_xy.index >= 31.015) & (df_xy.index <= 32.226)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 4"] = df_xy[(df_xy.index >= 10.001) & (df_xy.index <= 20.084)].idxmax(axis=0).to_frame().reset_index()[0]


df_maxs["Sample ID"] = df_maxs["Sample ID"].str.replace("2EXP", "4EXP")
df_maxs["Sample ID"] = df_maxs["Sample ID"].str.replace("3EXP", "4EXP")


# rename all files to 4EXP
for file in files:
    if "2EXP" in file:
        os.rename(os.path.join(directory, file), os.path.join(directory, file.replace("2EXP", "4EXP")))
    else:
        os.rename(os.path.join(directory, file), os.path.join(directory, file.replace("3EXP", "4EXP")))

df_maxs["Peak Ratio"] = df_maxs["Int. 1"] / (df_maxs["Int. 1"] + df_maxs["Int. 4"])
df_maxs.style.format({"Peak Ratio": "{:.2%}"})

# cm = plt.get_cmap("tab10")

# for _, row in df_maxs.iterrows():
#     sample_id = row["Sample ID"]
#     xy = df_xy[sample_id]
#     plt.figure(figsize=(6, 5))
#     plt.plot(df_xy.index, xy, label=sample_id, linewidth=0.5, color="black")

#     # annot_str = "".join([f'{row["2theta 1"]:.2f}, {row["Int. 1"]:.2f}\n', f'{row["2theta 2"]:.2f}, {row["Int. 2"]:.2f}\n', f'{row["2theta 3"]:.2f}, {row["Int. 3"]:.2f}\n', f'{row["2theta 4"]:.2f}, {row["Int. 4"]:.2f}\n'])
#     #list comp with label
#     annot_str = "".join([f'Peak {i}: {row[f"2theta {i}"]:.2f}, {row[f"Int. {i}"]:.2f}\n' for i in range(1, 5)])
#     # Adding vertical lines and annotating peaks with tuples of (2theta, Intensity)
#     for i in range(1, 5):
#         peak_x = row[f"2theta {i}"]
#         peak_y = row[f"Int. {i}"]
#         # plt.axvline(x=peak_x, color='grey', linestyle='--', alpha=0.5)
#         # plt.annotate(f'({peak_x:.2f}, {peak_y:.2f})', (peak_x, peak_y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=6)
#         #match color of annotation to line and offset each annotation
        
#         #peak_y = middel of plot
#         annot_y = max(xy) / 2
#         plt.axvline(x=peak_x, color=cm.colors[i-1], linestyle='-.', alpha=1.,linewidth=0.5)
#         # plot a small x at the peak
#         plt.scatter(peak_x, peak_y, marker='x', color=cm.colors[i-1], s=10)
#         plt.annotate(f'({peak_x:.2f}, {peak_y:.2f})', (peak_x, annot_y+i*annot_y/10),
#                      textcoords="offset points", xytext=(0,10), ha='left', fontsize=6, color=cm.colors[i-1])
    

#     # Adding peak ratio as a comment
#     plt.text(0.95, 0.95, r'Peak Ratio $\frac{1}{4+1}$' + f': {row["Peak Ratio"]:.2%}\n {annot_str}', transform=plt.gca().transAxes, 
#              horizontalalignment='right', verticalalignment='top', fontsize=6, bbox=dict(facecolor='white', alpha=0.5))

#     plt.xlabel('2theta')
#     plt.ylabel('Intensity')
#     plt.title(f'XRD of {sample_id}')
#     # plt.legend(loc='lower right', fontsize=6)
#     plt.grid(True)
#     plt.savefig(f'./XRD/xy_plot/{sample_id}.png', dpi=400)
#     plt.close()

,Sample ID,Int. 1,Int. 2,Int. 3,Int. 4,2theta 1,2theta 2,2theta 3,2theta 4,Peak Ratio
0,4EXP0,31.094000,15.312000,29.375000,101.198000,36.101700,34.224400,31.590200,11.757700,23.50%
1,4EXP100,31.146000,24.844000,22.708000,56.510000,36.162200,34.315200,31.650700,11.848500,35.53%
2,4EXP101,22.917000,11.875000,21.146000,3.385000,36.132000,34.224400,31.620400,10.758500,87.13%
3,4EXP10,77.031000,63.646000,50.260000,5.417000,36.132000,34.315200,31.590200,11.091500,93.43%
4,4EXP11,57.812000,56.146000,43.490000,5.260000,36.132000,34.285000,31.620400,10.485900,91.66%
5,4EXP12,37.708000,19.687000,36.354000,75.990000,36.071400,34.224400,31.559900,11.515400,33.17%
6,4EXP13,82.604000,70.833000,58.073000,5.260000,36.162200,34.315200,31.620400,11.636500,94.01%
7,4EXP14,8.021000,6.146000,7.552000,194.687000,36.071400,34.103300,31.045100,10.001500,3.96%
8,4EXP15,80.625000,66.146000,54.948000,5.729000,36.132000,34.285000,31.650700,11.242900,93.37%
9,4EXP16,5.990000,5.521000,6.771000,197.240000,35.798900,34.557500,31.408500,10.697900,2.95%


# Load synthesis data

In [15]:
# Load the data
synthesis_data = pd.read_excel(
    "./final/experiments_entry.xlsx", sheet_name="Clean data"
)

# calculate the std of pH
# synthesis_data["std"] = synthesis_data.groupby(["id"])["pH"].transform("std")
synthesis_data["Experiment"] = "4EXP" + synthesis_data["Experiment"].astype(str)
synthesis_data = synthesis_data.rename(columns={"Experiment": "Sample ID"})

synthesis_data.dropna(subset=["id"], inplace=True)

display(synthesis_data.columns)

synthesis_data = synthesis_data[
    [
        "Sample ID",
        "id",
        "repeat",
        "pH",
        "Cond",
        "C_ZnCl",
        "C_NaOH/C_ZnCl",
        "C_NaOH",
        "Q_AC",
        "Q_AIR",
        "N_ZnO",
        "End",
    ]
]

#sort by id and repeat

synthesis_data = synthesis_data.sort_values(by=["id", "repeat"])
synthesis_data

Index(['Sample ID', 'repeat', 'id', 'Q_AIR', 'Q_AC', 'Q_BD', 'C_Blk_ZnCl',
       'C_Blk_NaOH', 'C_ZnCl', 'C_NaOH/C_ZnCl', 'C_NaOH', 'Q_A', 'Q_B', 'Q_C',
       'Q_D', 'N_ZnO', 'End', 'batch', 'm_ct_1_dry', 'm_ct_2_dry', 'V_ct_calc',
       'V_ct_rxn', 'm_ct_full', 'm_ct_disp', 'pH', 'Cond', 'V_ct_q_1',
       'V_ct_q_2', 'V_ct_q_total', 'm_ct_q_1', 'm_ct_q_2', 'Cond_ct_q_1',
       'Cond_ct_q_2', 'Comments', 'XRD', 'm_ct_m_full', 'm_gv_empty',
       'V_gv_sus', 'm_gv_full', 'm_gv_disp', 'm_gv_powder', 'm_powder',
       'solid_gv%', 'm_ct_m_solid', 'solid_rxn_ct_1%'],
      dtype='object')

,Sample ID,id,repeat,pH,Cond,C_ZnCl,C_NaOH/C_ZnCl,C_NaOH,Q_AC,Q_AIR,N_ZnO,End
0,4EXP0,0,0,10.00,16.50,0.28,1.76,0.4928,4.03,2.32,0.135408,387.990060
12,4EXP12,0,1,10.95,17.22,0.28,1.76,0.4928,4.03,2.32,0.135408,387.990060
24,4EXP24,0,2,9.87,16.86,0.28,1.76,0.4928,4.03,2.32,0.135408,387.990060
1,4EXP1,1,0,13.04,72.65,0.66,2.59,1.7094,9.15,1.09,0.724680,238.032792
13,4EXP13,1,1,13.07,72.64,0.66,2.59,1.7094,9.15,1.09,0.724680,238.032792
...,...,...,...,...,...,...,...,...,...,...,...,...
98,4EXP98,32,1,9.93,30.71,0.49,2.02,0.9900,10.00,1.35,4.900000,228.000000
100,4EXP100,32,2,9.70,30.08,0.49,2.02,0.9900,10.00,1.35,4.900000,228.000000
97,4EXP97,33,0,12.59,17.09,0.10,2.81,0.2800,9.43,1.60,0.940000,234.530000
99,4EXP99,33,1,12.53,14.80,0.10,2.81,0.2800,9.43,1.60,0.940000,234.530000


# Pressure Data

In [16]:
pressure_df = pd.read_excel("./final/GAS-004-full-exp_perssure_data.xlsx")
pressure_df
#only keep the columns we need
# Unnamed: 0	experiment_number	pressure_a_mean	pressure_a_std	pressure_b_mean	pressure_b_std	pressure_c_mean	pressure_c_std	pressure_d_mean	pressure_d_std	pressure_air_mean	pressure_air_std	flowrate_air_mean	flowrate_air_std	_runtime_mean	_runtime_std

# a b air mean + std
pressure_df = pressure_df[["experiment_number", "pressure_a_mean", "pressure_a_std", "pressure_b_mean", "pressure_b_std", "pressure_air_mean", "pressure_air_std"]]
                           
pressure_df["Sample ID"] = "4EXP" + pressure_df["experiment_number"].astype(str)
pressure_df
#join with synthesis data to get id info
pressure_df.style.format({"pressure_a_mean": "{:.2f}", "pressure_a_std": "{:.2f}", "pressure_b_mean": "{:.2f}", "pressure_b_std": "{:.2f}", "pressure_air_mean": "{:.2f}", "pressure_air_std": "{:.2f}"}).background_gradient(cmap='Blues') 

,experiment_number,pressure_a_mean,pressure_a_std,pressure_b_mean,pressure_b_std,pressure_air_mean,pressure_air_std,Sample ID
0,0,0.19,0.20,2.78,0.03,8.23,0.00,4EXP0
1,1,0.67,0.10,5.35,0.02,4.05,0.01,4EXP1
2,2,0.42,0.08,4.09,0.04,6.94,0.00,4EXP2
3,3,0.54,0.14,4.89,0.02,6.05,0.00,4EXP3
4,4,0.39,0.05,4.44,0.16,5.34,0.01,4EXP4
5,5,0.37,0.09,4.66,0.02,7.02,0.00,4EXP5
6,6,0.49,0.09,4.05,0.04,4.67,0.00,4EXP6
7,7,0.14,0.15,3.49,0.01,8.25,0.00,4EXP7
8,8,0.21,0.08,3.76,0.01,7.36,0.00,4EXP8
9,9,0.40,0.14,4.34,0.03,5.56,0.00,4EXP9


# Process peak data

In [17]:
# import pandas as pd

# # Reading the Excel file
# df_peak = pd.read_excel("./final/peaks+scherrer.xlsx")

# # Extracting 'Sample ID' from 'Scan' column and renaming 'Index' column
# df_peak["Sample ID"] = df_peak["Scan"].str.split(".raw").str[0]
# df_peak = df_peak.rename(columns={"Index": "Peak Index"})

# # Selecting only necessary columns
# columns_needed = [
#     "Peak Index",
#     "Left Angle",
#     "Right Angle",
#     "Sample ID",
#     "Obs. Max",
#     "Gross Int.",
#     "Net Height",
#     "Net Area",
#     "d (Obs. Max)",
#     "Crystallite Size",
#     "S/N Ratio",
#     "P/B Ratio"
# ]
# df_peak = df_peak[columns_needed]

# # Splitting data based on 'Peak Index' and adding suffixes
# df_peaks = {}
# for i in range(1, 5):
#     df_peaks[i] = df_peak[df_peak["Peak Index"] == i].add_suffix(f"_{i}")

# # Merging dataframes on 'Sample ID' with appropriate suffixes
# df_merged = df_peaks[1]
# for i in range(2, 5):
#     df_merged = pd.merge(df_merged, df_peaks[i], left_on=f"Sample ID_{i-1}", right_on=f"Sample ID_{i}", how="outer")

# # Renaming columns for final DataFrame
# final_columns = [
#     f"Crystallite Size_{i}" for i in range(1, 5)
# ] + [
#     f"Net Height_{i}" for i in range(1, 5)
# ]
# final_columns.insert(0, "Sample ID_1")

# # df_peak_p = df_merged[final_columns]
# df_peak_p = df_merged
# df_peak_p = df_peak_p.rename(columns={"Sample ID_1": "Sample ID"})

# # # Calculating the ratio of 'Net Height_1' and 'Net Height_4'
# # df_peak_p["Peak Ratio opt"] = df_peak_p["Net Height_1"] /(df_peak_p["Net Height_4"] + df_peak_p["Net Height_1"])
# # display(df_peak)
# #drop all the columns with constant values
 
# df_peak_p = df_peak_p.loc[:, (df_peak_p != df_peak_p.iloc[0]).any()]

# display(df_peak_p)


# # import seaborn as sns
# # import matplotlib.pyplot as plt

# # # sns.pairplot(df_peak_p, hue="Sample ID", diag_kind="kde")

# Merge XRD and synthesis data

In [18]:
# color code each cell based its value relative to other cells in the same column contious gradient
import seaborn as sns

cm = sns.light_palette("purple", as_cmap=True)

df_list = [synthesis_data, 
        #    df_peak_p,
           df_maxs, topas_df, 
           pressure_df
           ]

df = pd.DataFrame(columns=["Sample ID"])
for df_ in df_list:
    df = pd.merge(df, df_, on="Sample ID", how="outer")

# df_xy.to_excel("./final/xy_data.xlsx")
df.to_excel("./final/XRD+synthsis_data.xlsx", index=False)

# only relevant columns
df = df[
    [
        "Sample ID",
        "id",
        "pH",
        "C_ZnCl",
        "C_NaOH/C_ZnCl",
        "C_NaOH",
        "Q_AC",
        "Q_AIR",
        "N_ZnO",
        "End",
        "Cond",
        "Aspect Ratio",
        "Peak Ratio",
        "00n Size",
        "Isotropic Size",
        # "2theta 1",
        # "2theta 2",
        # "2theta 3",
        # "2theta 4",
        # "Int. 1",
        # "Int. 2",
        # "Int. 3",
        # "Int. 4",
        # "Net Height_1",
        # "Net Height_2",
        # "Net Height_3",
        # "Net Height_4",
        "pressure_air_std",
        "pressure_b_std",
        "pressure_b_mean",
        "pressure_a_std",
        "pressure_a_mean",
    ]
]
df.dropna(how="any", axis=1, inplace=True)

# only where C_NaOH/C_ZnCl is bigger than 1.6
df = df[df["C_NaOH/C_ZnCl"] > 1.6]

#format all float columns cheeck for string columns

display(df
        .select_dtypes(include=["float64", "int64"])
        .style
        .background_gradient(cmap=cm, axis=0).format("{:.2f}"))


df_mean = df.select_dtypes(include=["float64", "int64"]).groupby("id").mean()


df_std = df.select_dtypes(include=["float64", "int64"]).groupby("id").std()
# replace id index with first sample id of each id
# display(df_mean.style.background_gradient(cmap=cm))

from scipy.stats import norm

def calculate_var(mean, std_dev, alpha=0.9):

    # Calculate the z-score for the given alpha level
    z_score = norm.ppf(alpha)

    # Calculate mVaR for each variable
    var = mean - z_score * std_dev
    return var


# df_std_rel["Sample ID"] = df.groupby("id").first()["Sample ID"]
# df_std_rel = df_std_rel.set_index("Sample ID")

# display(df_std.style.background_gradient(cmap=cm))
# merge both dataframes with suffix _mean and _std
df_mean_std = pd.merge(
    df_mean, df_std, left_index=True, right_index=True, suffixes=("_mean", "_std")
)

for col in df_mean.columns:
    df_mean_std[col + "_VaR"] = calculate_var(
        df_mean_std[col + "_mean"], df_mean_std[col + "_std"]
    )

#make separate df for VaR
df_VaR = df_mean_std.filter(like="VaR", axis=1)
display(df_mean.style.background_gradient(cmap=cm, axis=0).format("{:.2f}"))

# sort columns alphabetically
# df_mean_std = df_mean_std.reindex(sorted(df_mean_std.columns), axis=1)
df_mean_std["id_idx"] = df_mean_std.index
# drop all zerp columns
df_mean_std = df_mean_std.loc[:, (df_mean_std != 0).any(axis=0)]
# drop VaR columns if all VALUES are equal to mean columns VALUES
del df_mean_std["C_ZnCl_VaR"]
del df_mean_std["C_NaOH/C_ZnCl_VaR"]

df_mean_std["inital_sampling"] = df_mean_std["id_idx"] < 12
df["inital_sampling"] = df["id"] < 12
 
display(df_mean_std.style.background_gradient(cmap="Greens", axis=0).format("{:.2f}"))

,id,pH,C_ZnCl,C_NaOH/C_ZnCl,C_NaOH,Q_AC,Q_AIR,N_ZnO,End,Cond,Aspect Ratio,Peak Ratio,00n Size,Isotropic Size,pressure_air_std,pressure_b_std,pressure_b_mean,pressure_a_std,pressure_a_mean
0,0.00,10.00,0.28,1.76,0.49,4.03,2.32,0.14,387.99,16.50,2.61,0.24,36.42,13.98,0.00,0.03,2.78,0.20,0.19
1,0.00,10.95,0.28,1.76,0.49,4.03,2.32,0.14,387.99,17.22,4.15,0.33,59.19,14.25,0.00,0.03,2.74,0.22,0.19
2,0.00,9.87,0.28,1.76,0.49,4.03,2.32,0.14,387.99,16.86,3.66,0.12,65.26,17.85,0.00,0.03,2.76,0.17,0.19
3,1.00,13.04,0.66,2.59,1.71,9.15,1.09,0.72,238.03,72.65,2.24,0.85,24.75,11.04,0.01,0.02,5.35,0.10,0.67
4,1.00,13.07,0.66,2.59,1.71,9.15,1.09,0.72,238.03,72.64,2.34,0.94,28.60,12.24,0.01,0.02,5.26,0.10,0.64
5,1.00,13.03,0.66,2.59,1.71,9.15,1.09,0.72,238.03,73.17,2.30,0.94,32.59,14.16,0.01,0.02,5.34,0.13,0.71
9,3.00,13.06,0.51,2.75,1.40,7.89,1.66,0.48,256.88,62.80,2.26,0.94,24.61,10.90,0.00,0.02,4.89,0.14,0.54
10,3.00,13.03,0.51,2.75,1.40,7.89,1.66,0.48,256.88,61.62,2.27,0.93,28.06,12.34,0.00,0.02,4.82,0.14,0.54
11,3.00,13.02,0.51,2.75,1.40,7.89,1.66,0.48,256.88,62.03,2.59,0.93,36.37,14.07,0.00,0.03,4.86,0.17,0.54
15,5.00,13.46,0.93,3.17,2.95,5.14,1.95,0.57,330.12,125.64,2.28,0.95,23.41,10.28,0.00,0.02,4.66,0.09,0.37


,pH,C_ZnCl,C_NaOH/C_ZnCl,C_NaOH,Q_AC,Q_AIR,N_ZnO,End,Cond,Aspect Ratio,Peak Ratio,00n Size,Isotropic Size,pressure_air_std,pressure_b_std,pressure_b_mean,pressure_a_std,pressure_a_mean
id,,,,,,,,,,,,,,,,,,
0,10.27,0.28,1.76,0.49,4.03,2.32,0.14,387.99,16.86,3.47,0.23,53.62,15.36,0.00,0.03,2.76,0.20,0.19
1,13.05,0.66,2.59,1.71,9.15,1.09,0.72,238.03,72.82,2.29,0.91,28.65,12.48,0.01,0.02,5.32,0.11,0.67
3,13.04,0.51,2.75,1.40,7.89,1.66,0.48,256.88,62.15,2.37,0.93,29.68,12.44,0.00,0.03,4.86,0.15,0.54
5,13.43,0.93,3.17,2.95,5.14,1.95,0.57,330.12,126.32,2.28,0.94,25.74,11.31,0.00,0.02,4.65,0.10,0.37
7,11.66,0.10,2.27,0.23,5.81,2.31,0.07,305.89,10.10,3.93,0.14,68.73,19.78,0.00,0.02,3.48,0.12,0.15
9,13.28,0.77,2.99,2.30,6.07,1.54,0.56,297.92,102.88,2.36,0.94,27.16,11.54,0.00,0.03,4.39,0.20,0.42
10,13.16,0.98,2.00,1.96,9.99,2.20,1.17,228.11,84.91,2.51,0.94,29.46,11.76,0.00,0.01,7.04,0.38,0.68
11,12.70,0.39,2.45,0.96,4.88,1.20,0.23,341.31,36.44,2.68,0.89,37.29,14.14,0.00,0.05,2.64,0.21,0.28
12,13.42,0.88,3.09,2.72,9.45,1.23,8.32,234.29,121.94,2.27,0.92,22.51,9.90,0.00,0.06,6.63,0.20,0.62


,pH_mean,C_ZnCl_mean,C_NaOH/C_ZnCl_mean,C_NaOH_mean,Q_AC_mean,Q_AIR_mean,N_ZnO_mean,End_mean,Cond_mean,Aspect Ratio_mean,Peak Ratio_mean,00n Size_mean,Isotropic Size_mean,pressure_air_std_mean,pressure_b_std_mean,pressure_b_mean_mean,pressure_a_std_mean,pressure_a_mean_mean,pH_std,Cond_std,Aspect Ratio_std,Peak Ratio_std,00n Size_std,Isotropic Size_std,pressure_air_std_std,pressure_b_std_std,pressure_b_mean_std,pressure_a_std_std,pressure_a_mean_std,pH_VaR,C_NaOH_VaR,Q_AC_VaR,Q_AIR_VaR,N_ZnO_VaR,End_VaR,Cond_VaR,Aspect Ratio_VaR,Peak Ratio_VaR,00n Size_VaR,Isotropic Size_VaR,pressure_air_std_VaR,pressure_b_std_VaR,pressure_b_mean_VaR,pressure_a_std_VaR,pressure_a_mean_VaR,id_idx,inital_sampling
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10.27,0.28,1.76,0.49,4.03,2.32,0.14,387.99,16.86,3.47,0.23,53.62,15.36,0.00,0.03,2.76,0.20,0.19,0.59,0.36,0.79,0.10,15.20,2.16,0.00,0.00,0.02,0.03,0.00,9.52,0.49,4.03,2.32,0.14,387.99,16.40,2.47,0.10,34.14,12.59,0.00,0.03,2.73,0.16,0.19,0.00,1.00
1,13.05,0.66,2.59,1.71,9.15,1.09,0.72,238.03,72.82,2.29,0.91,28.65,12.48,0.01,0.02,5.32,0.11,0.67,0.02,0.30,0.05,0.06,3.92,1.57,0.00,0.00,0.05,0.02,0.03,13.02,1.71,9.15,1.09,0.72,238.03,72.43,2.23,0.84,23.62,10.46,0.00,0.01,5.26,0.09,0.63,1.00,1.00
3,13.04,0.51,2.75,1.40,7.89,1.66,0.48,256.88,62.15,2.37,0.93,29.68,12.44,0.00,0.03,4.86,0.15,0.54,0.02,0.60,0.19,0.01,6.05,1.59,0.00,0.00,0.03,0.02,0.00,13.01,1.40,7.89,1.66,0.48,256.88,61.38,2.13,0.92,21.93,10.40,0.00,0.02,4.82,0.13,0.53,3.00,1.00
5,13.43,0.93,3.17,2.95,5.14,1.95,0.57,330.12,126.32,2.28,0.94,25.74,11.31,0.00,0.02,4.65,0.10,0.37,0.03,0.59,0.04,0.02,2.93,1.46,0.00,0.01,0.05,0.02,0.01,13.39,2.95,5.14,1.95,0.57,330.12,125.56,2.23,0.92,21.98,9.45,0.00,0.02,4.59,0.08,0.36,5.00,1.00
7,11.66,0.10,2.27,0.23,5.81,2.31,0.07,305.89,10.10,3.93,0.14,68.73,19.78,0.00,0.02,3.48,0.12,0.15,1.70,2.78,1.59,0.12,4.92,8.72,0.00,0.00,0.02,0.03,0.01,9.47,0.23,5.81,2.31,0.07,305.89,6.54,1.90,-0.02,62.43,8.61,0.00,0.01,3.46,0.08,0.14,7.00,1.00
9,13.28,0.77,2.99,2.30,6.07,1.54,0.56,297.92,102.88,2.36,0.94,27.16,11.54,0.00,0.03,4.39,0.20,0.42,0.03,0.82,0.10,0.01,2.57,1.58,0.00,0.00,0.05,0.07,0.02,13.25,2.30,6.07,1.54,0.56,297.92,101.83,2.23,0.92,23.86,9.52,0.00,0.03,4.33,0.11,0.39,9.00,1.00
10,13.16,0.98,2.00,1.96,9.99,2.20,1.17,228.11,84.91,2.51,0.94,29.46,11.76,0.00,0.01,7.04,0.38,0.68,0.07,5.70,0.12,0.01,3.92,1.72,0.00,0.00,0.05,0.00,0.04,13.07,1.96,9.99,2.20,1.17,228.11,77.61,2.36,0.93,24.43,9.56,0.00,0.01,6.97,0.37,0.62,10.00,1.00
11,12.70,0.39,2.45,0.96,4.88,1.20,0.23,341.31,36.44,2.68,0.89,37.29,14.14,0.00,0.05,2.64,0.21,0.28,0.10,2.27,0.33,0.08,3.39,3.11,0.00,0.00,0.02,0.01,0.01,12.58,0.96,4.88,1.20,0.23,341.31,33.53,2.27,0.78,32.94,10.16,0.00,0.04,2.62,0.19,0.26,11.00,1.00
12,13.42,0.88,3.09,2.72,9.45,1.23,8.32,234.29,121.94,2.27,0.92,22.51,9.90,0.00,0.06,6.63,0.20,0.62,0.01,0.70,0.05,0.03,0.59,0.05,0.00,0.00,0.02,0.01,0.00,13.41,2.72,9.45,1.23,8.32,234.29,121.05,2.21,0.88,21.76,9.83,0.00,0.05,6.61,0.19,0.61,12.00,0.00


# Resample XRD Data

In [19]:

# apply moving average to the data so that the total number of points is 10% of the original data
window_size = 10


# Apply moving average. The function below applies a simple moving average on a Series.
def moving_average(series, window_size):
    return series.rolling(window=window_size, min_periods=1, center=True).mean()


# Apply moving average to each column in df_xy and downsample the data
df_xy_ma = pd.DataFrame()

low_res_selction = (df_xy.index > 40) | (df_xy.index < 20)

df_low_res_sect = df_xy.loc[low_res_selction]


for column in df_xy.columns:
    # Apply moving average
    ma_series = moving_average(df_low_res_sect[column], window_size)
    # Downsample the data by selecting every nth point, where n is the window size
    downsampled_series = ma_series[::window_size]
    df_xy_ma[column] = pd.concat(
        [df_xy.loc[~low_res_selction][column], downsampled_series]
    ).sort_index()

# use df_xy data from 20 to 40 2theta

# Adjust the index to match the downsampled data
df_xy_ma.index = pd.concat(
    [
        df_xy.loc[~low_res_selction].index.to_series(),
        downsampled_series.index.to_series(),
    ]
).sort_index()

df_xy_ma


C:\Users\queim\AppData\Local\Temp\ipykernel_7256\4105159979.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_xy_ma[column] = pd.concat(
C:\Users\queim\AppData\Local\Temp\ipykernel_7256\4105159979.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_xy_ma[column] = pd.concat(


,4EXP0,4EXP100,4EXP101,4EXP10,4EXP11,4EXP12,4EXP13,4EXP14,4EXP15,4EXP16,...,4EXP91,4EXP92,4EXP93,4EXP94,4EXP95,4EXP96,4EXP97,4EXP98,4EXP99,4EXP9
2theta,,,,,,,,,,,,,,,,,,,,,
10.0015,20.312600,7.8022,2.562400,4.1666,4.166600,17.8332,4.437600,182.010400,3.854400,160.260400,...,140.802000,3.333200,152.635400,4.2186,155.583200,10.677200,2.645800,13.583200,2.687400,4.093600
10.3043,18.614700,6.8906,2.718700,4.0051,3.974000,11.8228,4.192700,117.823000,4.057400,174.776000,...,138.140600,3.744700,156.770800,3.6563,166.385300,9.093700,2.677000,12.442600,2.713500,3.968800
10.6071,14.958200,6.9322,2.385600,3.9480,3.838500,8.1614,4.114600,65.150900,4.135400,191.463600,...,125.609300,3.588400,138.906300,4.1145,153.781100,7.984200,2.494800,10.713500,2.447900,3.921900
10.9098,16.078100,6.4634,2.604100,4.3542,3.802000,11.6353,3.682200,43.036300,4.573000,174.020800,...,89.614400,4.010200,96.354200,4.6771,110.140600,7.031300,2.296800,10.224000,2.744600,4.151000
11.2126,19.249700,7.6719,2.411500,4.4636,4.031100,14.8177,4.083500,33.156200,4.687500,129.729100,...,59.447900,3.619700,62.322900,5.8958,69.057200,7.770900,2.505200,11.687400,2.442700,4.343700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118.7018,1.171900,1.4323,0.812400,1.3542,1.260300,0.8490,1.411500,1.630000,1.406300,1.484400,...,1.520700,1.182200,1.171900,1.1666,1.369800,1.750000,0.619600,1.531000,0.724000,1.708300
119.0046,0.979100,1.4270,0.734300,1.4114,1.213500,0.8801,1.562400,1.546900,1.452900,1.385400,...,1.437400,1.031300,1.338500,1.1457,1.479200,1.640600,0.640600,1.427000,0.645800,1.776100
119.3074,1.166600,1.4634,0.718800,1.3594,1.244700,0.8280,1.432200,1.661500,1.562400,1.442600,...,1.328000,1.286300,1.239600,1.3122,1.255100,1.718700,0.573000,1.312500,0.750000,1.619800


# Explore the data (not aggregated)

In [20]:
import altair as alt
import pandas as pd
import panel as pn

alt.data_transformers.disable_max_rows()
pn.extension("vega")
def plot():
    # Dropdowns for selecting columns for X-axis, Y-axis, and Color
    x_axis = pn.widgets.Select(name="X-axis", options=list(df.columns), value="Peak Ratio")
    y_axis = pn.widgets.Select(
        name="Y-axis", options=list(df.columns), value="Aspect Ratio"
    )
    color = pn.widgets.Select(
        name="Color", options=list(df.columns), value="pH"
    )
    # Assuming 'id' is present in df for symbol encoding
    id_column = "id"  # Specify the column name for id if different

    brush = alt.selection_interval(name="brush")  # selection of type "interval"

    # Define a selection for interacting with the legend
    legend_selection = alt.selection_point(fields=[id_column], bind="legend")


    # Since this script includes dynamic interaction, ensure `filtered_table` function logic aligns with your data analysis goals
    def filtered_table(selection):
        if not selection:
            return df.iloc[:0]
        query = " & ".join(
            f"{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}"
            for col, crange in selection.items()
        )
        return df.query(query)


    def create_chart(x_axis, y_axis, color, id_column, legend_selection):
        chart = (
            alt.Chart(df)
            .mark_point()
            .encode(
                x=alt.X(f"{x_axis}:Q", scale=alt.Scale(zero=False)),
                y=alt.Y(f"{y_axis}:Q", scale=alt.Scale(zero=False)),
                color=alt.Color(
                    f"{color}:Q",
                    scale=alt.Scale(scheme="viridis"),
                    legend=alt.Legend(title=color),
                ),
                shape=alt.Shape(
                    f"{id_column}:N", legend=alt.Legend(title="ID Shapes")
                ),  # Encode shape by ID
                tooltip=[x_axis, y_axis, color, id_column, "Sample ID"],
                opacity=alt.condition(
                    legend_selection, alt.value(1), alt.value(0.2)
                ),  # Control opacity based on selection
                #marker size constant but big
                size=alt.value(100),
            )
            .add_params(legend_selection)
            .properties(width=600, height=600)
            .add_params(brush)
        )
        return chart


    vega_pane = pn.pane.Vega(debounce=10)


    @pn.depends(x_axis.param.value, y_axis.param.value, color.param.value, watch=True)
    def update_chart(x_axis, y_axis, color):
        vega_pane.object = create_chart(x_axis, y_axis, color, id_column, legend_selection)


    # Initialize chart
    update_chart(x_axis.value, y_axis.value, color.value)

    vega_pane_2 = pn.pane.Vega(debounce=10)
    bind = alt.selection_interval(bind="scales")


    def create_detail_chart(selected_experiments):
        if not selected_experiments:
            return (
                alt.Chart(pd.DataFrame())
                .mark_line()
                .encode(
                    x="2theta:Q",  # Assuming 2theta is your X-axis for the detailed chart
                    y="Value:Q",  # This will be the dynamically populated Y-axis based on selected experiments
                )
                .properties(width=600, height=300, title="XRD Plot")
            )

        # Melt df_xy to make it "long" for Altair plotting
        df_long = df_xy_ma.reset_index().melt(
            id_vars=["2theta"],
            value_vars=selected_experiments,
            var_name="Experiment",
            value_name="Intensity",
        )

        chart = (
            alt.Chart(df_long)
            .mark_line()
            .encode(
                x=alt.X("2theta:Q", title="2theta"),
                y=alt.Y("Intensity:Q", title="Intensity"),
                color="Experiment:N",
                tooltip=["2theta", "Intensity"],
            )
            .properties(width=600, height=300)
            .add_params(bind)
        )
        return chart


    # Assuming sem is initially defined as shown previously
    sem = pn.pane.Image("./XRD/xy_grouped_plot/id_0.png", width=600, height=400)
    # convert all .tif to .png
    import pathlib

    # from PIL import Image

    # for file in pathlib.Path("./XRD/SEM/").rglob("*.tif"):
    #     file = str(file)
    #     img = Image.open(file)
    #     img.save(file.replace(".tif", ".png"))


    # This function now updates 'sem' directly, without returning a new pane
    def update_image(exp_id):
        if len(exp_id) == 0:
            # Reset to a default image if no selection is made
            sem.object = ""
            return
        else:
            for e_id in exp_id:
                im_stub = f"MT-KBH-GAS-001-{e_id.split('EXP')[-1]}*.png"
                im_paths = list(pathlib.Path("./XRD/SEM/").glob(im_stub))

                if len(im_paths) > 0:
                    sem.object = im_paths[0]
                    return


    @pn.depends(vega_pane.selection.param.brush, watch=True)
    def update_detail_chart(brush_selection):
        if brush_selection:
            selected_exp_ids = filtered_table(brush_selection)["Sample ID"].tolist()
            selected_experiments = [
                col for col in df_xy_ma.columns if col in selected_exp_ids
            ]
            vega_pane_2.object = create_detail_chart(selected_experiments)
            update_image(
                selected_exp_ids
            )  # Call update_image to change the displayed image
        else:
            vega_pane_2.object = create_detail_chart([])
            update_image([])  # Reset or update the image when no selection is made


    # Ensure the initial state of the dashboard is correctly set up
    update_detail_chart(None)


    sidebar = pn.Column(
        x_axis,
        y_axis,
        color,
        sizing_mode="stretch_width",
    )

    main = pn.Row(
        pn.Column(
            pn.Row(
                vega_pane,
                pn.Column(
                    sem,
                    vega_pane_2,
                ),
            ),
            pn.pane.DataFrame(pn.bind(filtered_table, vega_pane.selection.param.brush)),
        ),
        sizing_mode="stretch_width",
    )
    
    return main, sidebar

main1, sidebar1 = plot()

template = pn.template.BootstrapTemplate(
    title="XRD Dashboard",
    header_background="#00aec9",
)
template.sidebar.append(sidebar1)
template.main.append(main1)
template.servable()

template.show()

# dashboard = pn.Column(
#     pn.Row(
#         vega_pane,
#         pn.Column(x_axis, y_axis, color, sem),
#         vega_pane_2,  # The second Vega pane
#     ),
#     pn.Column(
#         pn.pane.DataFrame(pn.bind(filtered_table, vega_pane.selection.param.brush)),
#         height=350,
#     ),
#     sizing_mode="stretch_width",
# ).servable()
# dashboard.show()

Launching server at http://localhost:54007


# Explore the data (aggregated)

In [21]:
import altair as alt
import pandas as pd
import panel as pn

pn.extension("vega")
alt.data_transformers.disable_max_rows()


def plot_agg():
    df_mean_std_v = df_mean_std.reset_index()
    # drop all the columns with constant values

    # Dropdowns for selecting columns for X-axis, Y-axis, and Color
    x_axis = pn.widgets.Select(
        name="X-axis", options=list(df_mean_std_v.columns), value="Peak Ratio_VaR"
    )
    y_axis = pn.widgets.Select(
        name="Y-axis", options=list(df_mean_std_v.columns), value="Aspect Ratio_VaR"
    )
    color = pn.widgets.Select(
        name="Color", options=list(df_mean_std_v.columns), value="pH_mean"
    )
    marker = pn.widgets.Select(
        name="Marker", options=list(df_mean_std_v.columns), value="Aspect Ratio_std"
    )
    
    color_scale = pn.widgets.Select(
        name="Color Scale",
        options=["viridis", "magma", "inferno", "plasma", "cividis", "blueorange", "category10", "tableau20"],
        value="viridis",
    )
    

    brush = alt.selection_interval(name="brush")  # selection of type "interval"

    def create_chart_agg(x_axis, y_axis, color, color_scale, marker, tooltip_cols=["id"]):
        chart = (
            alt.Chart(df_mean_std_v)
            .mark_point()
            .encode(
                x=alt.X(f"{x_axis}:Q", scale=alt.Scale(zero=False)),
                y=alt.Y(f"{y_axis}:Q", scale=alt.Scale(zero=False)),
                color=alt.condition(
                    brush,
                    alt.Color(f"{color}:Q", scale=alt.Scale(scheme=color_scale)),
                    alt.value("lightgray"),
                ),
                tooltip=[x_axis, y_axis, color, marker, *tooltip_cols],
                size=alt.Size(f"{marker}:Q", scale=alt.Scale(range=[100, 200])),
            )
            .properties(width=600, height=600)
            .add_params(brush)
        )
        return chart

    vega_pane = pn.pane.Vega(debounce=10)

    @pn.depends(x_axis.param.value, y_axis.param.value, color.param.value, color_scale.param.value, marker.param.value, watch=True)
    def update_chart_agg(x_axis, y_axis, color, color_scale, marker):
        vega_pane.object = create_chart_agg(x_axis, y_axis, color, color_scale, marker)

    def filtered_table(selection):
        if not selection:
            return df_mean_std_v.iloc[:0]
        query = " & ".join(
            f"{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}"
            for col, crange in selection.items()
        )
        return df_mean_std.query(query)

    # Initialize chart
    update_chart_agg(x_axis.value, y_axis.value, color.value, color_scale.value, marker.value)

    vega_pane_2 = pn.pane.Vega(debounce=10)
    bind = alt.selection_interval(bind="scales")

    def create_detail_chart_agg(selected_experiments):
        if not selected_experiments:
            return (
                alt.Chart(pd.DataFrame())
                .mark_line()
                .encode(
                    x="2theta:Q",  # Assuming 2theta is your X-axis for the detailed chart
                    y="Value:Q",  # This will be the dynamically populated Y-axis based on selected experiments
                )
                .properties(width=600, height=600, title="XRD Plot")
            )

        # Melt df_xy to make it "long" for Altair plotting
        df_long = df_xy_ma.reset_index().melt(
            id_vars=["2theta"],
            value_vars=selected_experiments,
            var_name="Experiment",
            value_name="Intensity",
        )

        chart = (
            alt.Chart(df_long)
            .mark_line()
            .encode(
                x=alt.X("2theta:Q", title="2theta"),
                y=alt.Y("Intensity:Q", title="Intensity"),
                color="Experiment:N",
                tooltip=["2theta", "Intensity"],
            )
            .properties(width=600, height=600)
            .add_params(bind)
        )
        return chart

    @pn.depends(vega_pane.selection.param.brush, watch=True)
    def update_detail_chart_agg(brush_selection):
        if brush_selection:
            selected_ids = filtered_table(brush_selection).index.tolist()

            # get corresponding experiments_ids from ids
            selected_exp_ids = df[df["id"].isin(selected_ids)]["Sample ID"].tolist()

            # Assuming the ids correspond to column names in df_xy or can be mapped to them
            selected_experiments = [
                col for col in df_xy_ma.columns if col in selected_exp_ids
            ]
            vega_pane_2.object = create_detail_chart_agg(selected_experiments)
        else:
            vega_pane_2.object = create_detail_chart_agg([])

    # Initial call to update the detail chart with no selection
    update_detail_chart_agg(None)

    main = pn.Column(
        pn.Row(
            vega_pane,
            pn.Column(x_axis, y_axis, color, marker),
            vega_pane_2,  # Add the second Vega pane here
        ),
        pn.Column(
            pn.pane.DataFrame(
                pn.bind(filtered_table, vega_pane.selection.param.brush),
                float_format=lambda x: f"{x:.3f}",
            ),
            height=350,
        ),
        sizing_mode="stretch_width",
    ).servable()

    sidebar = pn.Column(color_scale, sizing_mode="stretch_width")

    return main, sidebar


main_agg, sidebar_agg = plot_agg()

template_agg = pn.template.BootstrapTemplate(
    title="XRD Dashboard Agg", header_background="#00aec9"
)
template_agg.sidebar.append(sidebar_agg)
template_agg.main.append(main_agg)
template_agg.servable()

template_agg.show()

Launching server at http://localhost:54009


AssertionError: 

2024-03-11 16:37:13,751 ERROR: panel.reactive - Callback failed for object named "X-axis" changing property {'value': 'C_NaOH/C_ZnCl'} 
Traceback (most recent call last):
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\param\parameterized.py", line 2278, in update
    restore = self_._update(*args, **kwargs)
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\param\parameterized.py", line 2318, in _update
    self_._batch_call_watchers()
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\param\parameterized.py", line 2509, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site-packages\param\parameterized.py", line 2471, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "c:\Users\queim\micromambaenv\envs\mobo\lib\site

# Grouped XRD Plot

In [ ]:


import os
import pandas as pd
import matplotlib.pyplot as plt

# change resolution of plot
from matplotlib import rcParams
rcParams["font.size"] = 12
rcParams["font.family"] = "serif"
plt.rcParams['figure.dpi'] = 400
plt.rcParams["mathtext.fontset"] = "stix"

# Directory containing your raw files
directory = "./final/xy/"

# List all raw files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.xy')]  # Adjust the extension if necessary

# Analyzing each file
dict_xy = {
    "2theta": [],
}

file = files[0]
file_path = os.path.join(directory, file)
df_ = pd.read_csv(file_path, sep="\s+", skiprows=1, names=["2theta", "intensity"], dtype=float)
dict_xy["2theta"].extend(df_["2theta"])

thetha2 = []
for file in files:
    file_path = os.path.join(directory, file)
    df_ = pd.read_csv(file_path, sep="\s+", skiprows=1, names=["2theta", "intensity"], dtype=float)
    file_name = file.split("_exported.")[0]
    dict_xy[file_name] = df_["intensity"].tolist()
    
df_xy = pd.DataFrame(dict_xy).set_index("2theta")



# [35.496 ° - 36.859 °]
# [34.012 ° - 34.648 °]
# [31.015 ° - 32.226 °]
# [10.001 ° - 20.084 °]
#calculate the max intensity in each range

df_maxs = df_xy[(df_xy.index >= 35.496) & (df_xy.index <= 36.859)].max(axis=0).to_frame().reset_index().rename(columns={"index": "Sample ID", 0: "Int. 1"}) 
df_maxs["Int. 2"] = df_xy[(df_xy.index >= 34.012) & (df_xy.index <= 34.648)].max(axis=0).to_frame().reset_index()[0]
df_maxs["Int. 3"] = df_xy[(df_xy.index >= 31.015) & (df_xy.index <= 32.226)].max(axis=0).to_frame().reset_index()[0]
df_maxs["Int. 4"] = df_xy[(df_xy.index >= 10.001) & (df_xy.index <= 20.084)].max(axis=0).to_frame().reset_index()[0]

df_maxs["2theta 1"] = df_xy[(df_xy.index >= 35.496) & (df_xy.index <= 36.859)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 2"] = df_xy[(df_xy.index >= 34.012) & (df_xy.index <= 34.648)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 3"] = df_xy[(df_xy.index >= 31.015) & (df_xy.index <= 32.226)].idxmax(axis=0).to_frame().reset_index()[0]
df_maxs["2theta 4"] = df_xy[(df_xy.index >= 10.001) & (df_xy.index <= 20.084)].idxmax(axis=0).to_frame().reset_index()[0]


df_maxs["Peak Ratio"] = df_maxs["Int. 1"] / (df_maxs["Int. 1"] + df_maxs["Int. 4"])
df_maxs.style.format({"Peak Ratio": "{:.2%}"})

cm = plt.get_cmap("tab10")
df
df_mean_std
# for _, row in df_maxs.iterrows():
for id in df["id"].unique():
    
    sample_list = df[df["id"] == id]["Sample ID"].tolist()
    
    plt.figure(figsize=(6, 5))    
    line_styles = ["-", "--", "-.", ":"]
    # for idx, row in df_maxs[df_maxs["Sample ID"].isin(sample_list)].iterrows():
    #     sample_id = row["Sample ID"]
    df_slice = df[df["Sample ID"].isin(sample_list)]
    df_agg = df_mean_std[df_mean_std.index == id]
    for idx, sample_id in enumerate(sample_list):
        row = df_maxs[df_maxs["Sample ID"] == sample_id].iloc[0]
        xy = df_xy[sample_id]
        plt.plot(df_xy.index, xy, label=sample_id, linewidth=0.5, linestyle=line_styles[idx], color=cm.colors[idx])

        # annot_str = "".join([f'{row["2theta 1"]:.2f}, {row["Int. 1"]:.2f}\n', f'{row["2theta 2"]:.2f}, {row["Int. 2"]:.2f}\n', f'{row["2theta 3"]:.2f}, {row["Int. 3"]:.2f}\n', f'{row["2theta 4"]:.2f}, {row["Int. 4"]:.2f}\n'])
        #list comp with label
        # Adding vertical lines and annotating peaks with tuples of (2theta, Intensity)
        for i in range(1, 5):
            peak_x = row[f"2theta {i}"]
            peak_y = row[f"Int. {i}"]
            # plt.axvline(x=peak_x, color='grey', linestyle='--', alpha=0.5)
            # plt.annotate(f'({peak_x:.2f}, {peak_y:.2f})', (peak_x, peak_y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=6)
            #match color of annotation to line and offset each annotation
            
            #peak_y = middel of plot
            annot_y = max(xy) / 2
            plt.axvline(x=peak_x,
                        color="black", linestyle='-.', alpha=.8,linewidth=0.3)
            # plot a small x at the peak
            plt.scatter(peak_x, peak_y, marker='x', color=cm.colors[idx], s=10)
            plt.annotate(f'({peak_x:.2f}, {peak_y:.2f})', (peak_x, annot_y+i*annot_y/10),
                        textcoords="offset points", xytext=(0,10), ha='left', fontsize=6, color=cm.colors[idx])
        

        text = f'Peak Ratio: {df_agg["Peak Ratio_mean"].values[0]:.2%} ± {df_agg["Peak Ratio_std"].values[0]:.2%}\n '
        text += f'Aspect Ratio: {df_agg["Aspect Ratio_mean"].values[0]:.2f} ± {df_agg["Aspect Ratio_std"].values[0]:.2f}\n '
        text += f'pH: {df_agg["pH_mean"].values[0]:.2f} ± {df_agg["pH_std"].values[0]:.2f}\n '
        text += f'C$_{{ZnCl}}$: {df_agg["C_ZnCl_mean"].values[0]:.2f}\n'
        text += f'C$_{{NaOH/C_ZnCl}}$: {df_agg["C_NaOH/C_ZnCl_mean"].values[0]:.2f}'
        # Adding peak ratio as a comment
        plt.text(0.95, 0.95, text, transform=plt.gca().transAxes, 
                horizontalalignment='right', verticalalignment='top', fontsize=6, bbox=dict(facecolor='white', alpha=0.5))

    #legend mapping sample id to line style color
    plt.legend(loc='lower right', fontsize=6)
     
    
    plt.xlabel('2theta')
    plt.ylabel('Intensity')
    plt.title(f'XRD of {id} Samples: {sample_list}')
    # plt.legend(loc='lower right', fontsize=6)
    plt.grid(True)
    # plt.savefig(f'./XRD/xy_grouped_plot/id_{id}.png', dpi=400)
    #svg for vector graphics
    # plt.savefig(f'./XRD/xy_grouped_plot/id_{id}.svg', dpi=400)
    plt.close()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# df = df[df["Cluster"] == 1]
df_sub = df_mean_std[
    [
        "id",
        "pH",
        "C_ZnCl",
        # "C_NaOH/C_ZnCl",
        # "C_NaOH",
        # "solid_rxn_ct_1%",
        # "Crystallite Size_1",
        "Peak Ratio",
        "Aspect Ratio",
        "pressure_air_std",
        "pressure_b_std",
        "pressure_b_mean",
        
        # "%-Crystallinity",
        # "Component_Concentration_ZnO",
        # "Cluster",
        # "XRD",
        
    ]
]

#id bigger than 11

df_sub = df_sub[df_sub["id"] > 11]

# sns.pairplot(df[df["Cluster"] == 1], kind="kde")

KeyError: "None of [Index(['id', 'pH', 'C_ZnCl', 'Peak Ratio', 'Aspect Ratio', 'pressure_air_std',\n       'pressure_b_std', 'pressure_b_mean'],\n      dtype='object')] are in the [columns]"

In [ ]:
dff = df_mean_std.select_dtypes(include=['float64', 'int64'])
df_mean = dff.groupby(["id"]).mean()
df_std = dff.groupby(["id"]).std()
df_mean_std = pd.merge(df_mean, df_std, on="id", suffixes=("_mean", "_std"))
#color id
# sns.pairplot(df_mean,)

In [ ]:
df_std = df_sub.groupby(["id"]).std()
df_mean_std = pd.merge(df_mean, df_std, on="id", suffixes=("_mean", "_std"))
# sns.pairplot(df_mean_std)

In [ ]:
# only where the cluster is 1
# sns.pairplot(df_mean_std[df_mean_std["pH_mean"] > 11])

# Find Correlations between synthesis and XRD data

In [ ]:
import seaborn as sns
%matplotlib inline

#where pH is greater than 11
df_corr = df_mean_std[df_mean_std["pH"] > 11].select_dtypes(include="float").corr()
sns.heatmap(df_corr, annot=True,cmap="coolwarm", annot_kws={"size": 7})